from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/ColabNotebooks/ComplexNetworks/practice-2

In [ ]:
import networkx as nx
from datasets import zachary_dataset
from pprint import pprint
from sklearn.cluster import SpectralClustering, KMeans
from sklearn import metrics
import numpy as np

In [ ]:
data = zachary_dataset()

In [ ]:
# ایجاد یک گراف (شبکه) جدید
G = nx.Graph()

In [ ]:
G.add_edges_from(data)

In [ ]:
# values_ by Node
values_ = {}
node_to_cluster = {}

In [ ]:
# محاسبه ماتریس مجاورت گراف
adjacency_matrix = nx.to_numpy_array(G)

In [ ]:
# k-path centrality
def k_path_centrality(G, k):
    centrality = {}
    nodes = list(G.nodes())
    for node in nodes:
        total_paths = 0
        for source in nodes:
            if source != node:
                paths = list(
                    nx.all_simple_paths(G, source=source, target=node, cutoff=k)
                )
                total_paths += len(paths)
        centrality[node] = total_paths
    return centrality

In [ ]:
# closeness centrality
closeness_centrality = nx.closeness_centrality(G)

In [ ]:
# betweenness_centrality
betweenness_centrality = nx.betweenness_centrality(G)

In [ ]:
# neighbor_centrality
neighbor_centrality = {}

In [ ]:
# eigenvector_centrality
eigenvector_centrality = nx.eigenvector_centrality(G)

In [ ]:
# k-path centrality
k_path_centrality_ = k_path_centrality(G, k=3)

In [ ]:
# page rank
pagerank = nx.pagerank(G)

In [ ]:
# closeness centrality
for node, centrality in closeness_centrality.items():
    # print(f"Node: {node}: closeness_centrality: {centrality}", end="\n")
    values_[node] = {"closeness_centrality": centrality}

In [ ]:
# betweenness_centrality
for node, centrality in betweenness_centrality.items():
    # print(f"Node: {node}: betweenness_centrality: {centrality}", end="\n")
    values_[node]["betweenness_centrality"] = centrality

In [ ]:
# eigenvector_centrality
for node, centrality in eigenvector_centrality.items():
    # print(f"Node: {node}: eigenvector_centrality: {centrality}", end="\n")
    values_[node]["eigenvector_centrality"] = centrality

In [ ]:
# Neighbor-based centrality
for node in G.nodes():
    neighbor_count = len(list(G.neighbors(node)))
    neighbor_centrality[node] = neighbor_count

In [ ]:
for node, centrality in neighbor_centrality.items():
    # print(f"Node {node}: neighbor_centrality{centrality}")
    values_[node]["neighbor_centrality"] = centrality

In [ ]:
# k_path_centrality
for node, centrality in k_path_centrality_.items():
    # print(f"Node {node}: k_path_centrality: {centrality}")
    values_[node]["k_path_centrality"] = centrality

In [ ]:
# نمایش PageRank برای هر گره
for node, rank in pagerank.items():
    # print(f'گره {node}: PageRank = {rank:.4f}')
    values_[node]["PageRank"] = f"{rank:.4f}"

In [ ]:
# Spectral clustering
num_clusters = 2  # تعداد خوشه‌ها
clustering = SpectralClustering(n_clusters=num_clusters, affinity='rbf', random_state=0)
Spectral_labels = clustering.fit_predict(adjacency_matrix)
## Result 
for node, label in enumerate(Spectral_labels):
    # print(f'گره {node + 1} در خوشه {label + 1}')
    node_to_cluster[node+1] = label + 1 # used for modularity
    values_[node+1]["spectral_cluster"] = label + 1

In [ ]:
# K-Means clustering
## تعداد خوشه‌ها که می‌خواهید تعیین کنید
num_clusters = 2

In [ ]:
## ایجاد یک مدل 
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
## اجرای الگوریتم K-Means بر روی داده
kmeans.fit(data)
## دسته‌بندی داده‌ها به خوشه‌ها
labels = kmeans.labels_
## مراکز خوشه‌ها
cluster_centers = kmeans.cluster_centers_
##  Results
print(len(labels))
for i, label in enumerate(labels):
    # print(f'point {data[i][0]} in cluster: {label + 1}')
    if "kmeans" not in (values_[data[i][0]]):
        values_[data[i][0]]["kmeans"] = [{"point":data[i],"cluster":label + 1}]
    else:
        values_[data[i][0]]["kmeans"] += [{"point":data[i],"cluster":label + 1}]

Modularity using Spectral Clustering
# محاسبه معیار Modularity
print(G.nodes())
print(node_to_cluster)
# modularity = nx.community.modularity(G, data, weight='weight')

## نمایش معیار Modularity
print(f'Modularity: {modularity:.4f}')

In [ ]:
pprint(values_)